In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import math
import numpy as np
import os
import cv2
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from CustomDataset import CustomDataset
import matplotlib.pyplot as plt
torch.manual_seed(0)

torch.cuda.empty_cache()

In [2]:
#Initializations
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.segmentation.fcn_resnet50(pretrained=True).to(device)

#has to normalize data the same way the pretrained images were
normalize = transforms.Normalize(mean=[0.485,0.456,0.406],
                        std=[0.229,0.224,0.225])
data_transforms = {
    'train':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.ToTensor(),
        normalize
    ]),
}
dataset = CustomDataset(target_type = 'poly')

TRAIN_SIZE= math.floor(dataset.__len__()*0.75)
TEST_SIZE = dataset.__len__() - TRAIN_SIZE
trainset, testset = random_split(dataset,[TRAIN_SIZE,TEST_SIZE])


traindata_loader = DataLoader(trainset, batch_size=5, shuffle=True)
testdata_loader = DataLoader(testset, batch_size=5, shuffle=True)
# data_loader = DataLoader(dataset, batch_size=10, shuffle=True)
image_datasets = {
    'train': 
        TRAIN_SIZE,
    'validation':
        TEST_SIZE
}

dataloaders = {
    'train':
        traindata_loader,
    'validation':
        testdata_loader
}

for name,param in model.named_parameters():
    if 'FCNHead' not in name:
        param.requires_grad = False  
print(model.classifier[4])

model.classifier[4] = nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1)).to(device)

#model.fc = nn.Sequential(
#               nn.Linear(2048, 128),
#               nn.ReLU(inplace=True),
#               nn.Linear(128, 2)).to(device)
params_to_update = []
for param in model.parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        
# loss_fn = nn.CrossEntropyLoss()
loss_fn = nn.NLLLoss()
optim = optim.Adam((model.classifier[4].parameters()))
print(model.classifier[4])

/opt/anaconda-2022.05/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/anaconda-2022.05/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Conv2d(512, 21, kernel_size=(1, 1), stride=(1, 1))
Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))


In [3]:
for batch, (inputs,labels) in enumerate(dataloaders['train']):
# for inputs,labels in enumerate(dataloaders[phase]):
    # inputs = inputs.to(device)
    # labels = labels.to(device)
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([5, 3, 960, 1280])
torch.Size([5, 960, 1280])


In [4]:
def train_model(model, criterion, optimizer, num_epochs=3, evaluate = False):
    epoch_loss_list = []
    epoch_acc_list = []
    phases = ['train', 'validation']
    if evaluate == True:
        phases = ['validation']
    for epoch in range(num_epochs):
        for phase in phases:
            print("Currently in the: ", phase," phase")
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            total = 0
            running_corrects = 0
            acc = 0
            for batch, (inputs,labels) in enumerate(dataloaders[phase]):
            # for inputs,labels in enumerate(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # print(inputs.dtype,labels.dtype)
                # DON'T USE ARGMAX HERE
                # labels = torch.argmax(labels, dim=1)
                outputs = model(inputs)["out"]
                # print(labels.max, labels.min)
                
                loss = criterion(outputs, labels.long())

                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                total += labels.nelement()
                running_corrects += preds.eq(labels.data).sum().item()
                ## TODO
                # mean IoU
                # FP, FN
                acc = running_corrects/total
            # Plotting a random image at the end of all epochs
            plt1 = plt.figure()
            plt1 = plt.imshow(preds.cpu()[0])
            plt2 = plt.figure()
            plt2 = plt.imshow(labels.data.cpu()[0])
            # Calculating loss and acc
            epoch_loss = running_loss / image_datasets[phase]
            epoch_loss_list.append(epoch_loss)
            epoch_acc = acc / image_datasets[phase]
            epoch_acc_list.append(epoch_acc)
            
            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
    return model, epoch_loss_list, epoch_acc_list


In [ ]:
model_trained, epoch_loss_list, epoch_acc_list = train_model(model, loss_fn, optim, num_epochs=50)
#!mkdir models
#!mkdir models/pytorch
#torch.save(model_trained.state_dict(), 'models/pytorch/weights6.h5')

Currently in the:  train  phase
null: ./data/public-annotation/grayfoxMay2011.json
null: ./data/public-annotation/P1010547%20%281%29.json
null: ./data/public-annotation/395%20Owl.json
null: ./data/public-annotation/A_Bighorn_BadwaterRoad_MM_35_20121001.json
null: ./data/public-annotation/P1010851.json
train loss: -9.0440, acc: 0.0011
Currently in the:  validation  phase
null: ./data/public-annotation/P1000576.json
null: ./data/public-annotation/20180619_112554.json


In [ ]:
model = torchvision.models.segmentation.fcn_resnet50(pretrained=True)
model.classifier[4] = nn.Conv2d(512, 2, kernel_size=(1, 1), stride=(1, 1))
model.load_state_dict(torch.load('models/pytorch/weights5.h5'))
model.eval();

Evaluation

In [ ]:
model_trained, epoch_loss_list, epoch_acc_list = train_model(model, loss_fn, optim, num_epochs=1, evaluate = True)